<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [8]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [9]:
import folium
import pandas as pd

In [10]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [15]:
## Task 1: Mark all launch sites on a map

import folium

# Center the map on the mean latitude/longitude of all sites
site_map = folium.Map(
    location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()],
    zoom_start=5
)

# Add a Circle and a Marker for each launch site
for _, row in launch_sites_df.iterrows():
    lat, lon = float(row['Lat']), float(row['Long'])
    name = str(row['Launch Site'])

    # Circle to visualize the area
    folium.Circle(
        location=[lat, lon],
        radius=1000,            # meters
        color='blue',
        weight=2,
        fill=True,
        fill_opacity=0.3
    ).add_to(site_map)

    # Marker labeled with site name (hover + click)
    folium.Marker(
        location=[lat, lon],
        popup=name,
        tooltip=name,
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [13]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [14]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [23]:
def create_launch_sites_map():
    """Create a map with all SpaceX launch sites marked"""
    # Create a folium map centered around the average location of launch sites
    site_map = folium.Map(
        location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], 
        zoom_start=5
    )
    
    # Add each launch site as a circle marker
    for index, row in launch_sites_df.iterrows():
        # Create circle marker for each launch site
        folium.Circle(
            location=[row['Lat'], row['Long']],
            radius=1000,  # radius in meters
            color='blue',
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(row['Launch Site']),
            tooltip=row['Launch Site']
        ).add_to(site_map)
        
        # Add text marker for launch site name
        folium.Marker(
            [row['Lat'], row['Long']],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12; color:#0066cc;"><b>{row["Launch Site"]}</b></div>'
            )
        ).add_to(site_map)
    
    return site_map

In [16]:
def create_launch_sites_map():
    """Create a map with all SpaceX launch sites marked"""
    # Create a folium map centered around the average location of launch sites
    site_map = folium.Map(
        location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], 
        zoom_start=5
    )
    
    # Add each launch site as a circle marker
    for index, row in launch_sites_df.iterrows():
        # Create circle marker for each launch site
        folium.Circle(
            location=[row['Lat'], row['Long']],
            radius=1000,  # radius in meters
            color='blue',
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(row['Launch Site']),
            tooltip=row['Launch Site']
        ).add_to(site_map)
        
        # Add text marker for launch site name
        folium.Marker(
            [row['Lat'], row['Long']],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12; color:#0066cc;"><b>{row["Launch Site"]}</b></div>'
            )
        ).add_to(site_map)
    
    return site_map

An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [15]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [22]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [24]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def create_launch_sites_map():
    """Create a map with all SpaceX launch sites marked"""
    # Create a folium map centered around the average location of launch sites
    site_map = folium.Map(
        location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], 
        zoom_start=5
    )
    
    # Add each launch site as a circle marker
    for index, row in launch_sites_df.iterrows():
        # Create circle marker for each launch site
        folium.Circle(
            location=[row['Lat'], row['Long']],
            radius=1000,  # radius in meters
            color='blue',
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(row['Launch Site']),
            tooltip=row['Launch Site']
        ).add_to(site_map)
        
        # Add text marker for launch site name
        folium.Marker(
            [row['Lat'], row['Long']],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12; color:#0066cc;"><b>{row["Launch Site"]}</b></div>'
            )
        ).add_to(site_map)
    
    return site_map

spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [26]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

<class 'NameError'>: name 'marker' is not defined

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


In [32]:
# 1) Create marker_color column: class 1 -> green, class 0 -> red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# 2) Plot launches with a MarkerCluster
from folium.plugins import MarkerCluster

launch_map = folium.Map(
    location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()],
    zoom_start=4
)

marker_cluster = MarkerCluster().add_to(launch_map)

for _, r in spacex_df.iterrows():
    lat, lon = float(r['Lat']), float(r['Long'])
    color = r['marker_color']
    site = r.get('Launch Site', '')
    outcome = 'Success' if r['class'] == 1 else 'Failure'

    folium.Marker(
        location=[lat, lon],
        tooltip=f"{site} — {outcome}",
        popup=folium.Popup(f"<b>Site:</b> {site}<br><b>Outcome:</b> {outcome}", max_width=250),
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

launch_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [33]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [52]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [43]:
# Example: pick one site from launch_sites_df (e.g., first row)
site_row = launch_sites_df.iloc[0]
site_lat, site_lon = float(site_row['Lat']), float(site_row['Long'])
site_name = str(site_row['Launch Site'])

# Replace these with the coastline point you selected on the map
target_lat, target_lon = 28.563, -80.571  # example coords (Cape Canaveral coast-ish)

# Draw the line
line = PolyLine(
    locations=[(site_lat, site_lon), (target_lat, target_lon)],
    weight=3
)

proximity_map = folium.Map(location=[site_lat, site_lon], zoom_start=8)
folium.Marker([site_lat, site_lon], tooltip=site_name).add_to(proximity_map)
folium.Marker([target_lat, target_lon], tooltip="Selected coastline point").add_to(proximity_map)
line.add_to(proximity_map)

proximity_map

<class 'NameError'>: name 'PolyLine' is not defined

In [36]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [37]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [54]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on earth"""
    # Approximate radius of earth in km
    R = 6373.0
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

def add_proximity_analysis(site_map, launch_site_coords, proximity_coords, proximity_name, color='red'):
    """Add proximity analysis to the map with distance calculation and line"""
    # Calculate distance
    distance = calculate_distance(
        launch_site_coords[0], launch_site_coords[1],
        proximity_coords[0], proximity_coords[1]
    )
    
    # Create distance marker
    distance_marker = folium.Marker(
        proximity_coords,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:{color};"><b>{proximity_name}<br>{distance:.2f} KM</b></div>'
        ),
        popup=f"{proximity_name}: {distance:.2f} km from launch site"
    )
    
    # Create line between launch site and proximity point
    line = folium.PolyLine(
        locations=[launch_site_coords, proximity_coords],
        weight=2,
        color=color,
        opacity=0.8
    )
    
    # Add to map
    site_map.add_child(distance_marker)
    site_map.add_child(line)
    
    return distance

def create_complete_analysis_map():
    """Create complete map with launch sites, success/failure markers, and proximity analysis"""
    # Create base map centered on NASA JSC
    site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
    
    # Add mouse position control
    formatter = "function(num) {return L.Util.formatNum(num, 5);};"
    mouse_position = MousePosition(
        position='topright',
        separator=' Long: ',
        empty_string='NaN',
        lng_first=False,
        num_digits=20,
        prefix='Lat:',
        lat_formatter=formatter,
        lng_formatter=formatter,
    )
    site_map.add_child(mouse_position)
    
    # Add NASA JSC as reference point
    folium.Circle(
        nasa_coordinate, 
        radius=1000, 
        color='#d35400', 
        fill=True
    ).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)
    
    folium.Marker(
        nasa_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>NASA JSC</b></div>'
        )
    ).add_to(site_map)
    
    # Add launch site circles
    for index, row in launch_sites_df.iterrows():
        folium.Circle(
            location=[row['Lat'], row['Long']],
            radius=1000,
            color='blue',
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(row['Launch Site']),
            tooltip=row['Launch Site']
        ).add_to(site_map)
    
    # Add success/failure markers
    spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
    marker_cluster = MarkerCluster()
    
    for index, record in spacex_df.iterrows():
        marker = folium.Marker(
            [record['Lat'], record['Long']],
            icon=folium.Icon(color='white', icon_color=record['marker_color']),
            popup=folium.Popup(f"{record['Launch Site']}<br>Success: {'Yes' if record['class'] == 1 else 'No'}")
        )
        marker_cluster.add_child(marker)
    
    site_map.add_child(marker_cluster)
    
    # Example proximity analysis for KSC LC-39A
    # You would replace these coordinates with actual coordinates found using MousePosition
    ksc_coords = [28.573255, -80.646895]
    
    # Example coastline point (you'd find this using MousePosition on the map)
    coastline_coords = [28.56367, -80.57163]  # Example coordinates
    coastline_distance = add_proximity_analysis(
        site_map, ksc_coords, coastline_coords, "Coastline", color='blue'
    )
    
    # Example city point (you'd find this using MousePosition on the map)
    city_coords = [28.52861, -80.68333]  # Example coordinates for nearby city
    city_distance = add_proximity_analysis(
        site_map, ksc_coords, city_coords, "Nearest City", color='purple'
    )
    
    # Example highway point (you'd find this using MousePosition on the map)
    highway_coords = [28.61111, -80.60417]  # Example coordinates for highway
    highway_distance = add_proximity_analysis(
        site_map, ksc_coords, highway_coords, "Highway", color='orange'
    )
    
    return site_map

create_complete_analysis_map(location=nasa_coordinate, zoom_start=5)


<class 'NameError'>: name 'nasa_coordinate' is not defined

In [46]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

<class 'NameError'>: name 'coordinates' is not defined

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [47]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
def analyze_launch_sites():
    """Analyze launch site locations and success rates"""
    
    print("SpaceX Launch Site Analysis")
    print("=" * 50)
    
    # Launch site locations analysis
    print("\n1. LAUNCH SITE LOCATIONS:")
    for index, row in launch_sites_df.iterrows():
        print(f"   {row['Launch Site']}: Lat {row['Lat']:.3f}, Long {row['Long']:.3f}")
    
    # Success rate analysis by site
    print("\n2. SUCCESS RATES BY LAUNCH SITE:")
    success_rates = spacex_df.groupby('Launch Site')['class'].agg(['count', 'sum', 'mean']).round(3)
    success_rates.columns = ['Total Launches', 'Successful Launches', 'Success Rate']
    print(success_rates)
    
    # Geographic insights
    print("\n3. GEOGRAPHIC INSIGHTS:")
    print("   - Proximity to Equator: All sites are relatively close to the equator")
    print("     (between 28°N and 35°N latitude) which is advantageous for launches")
    print("   - Coastal Location: All sites are very close to coastlines")
    print("     (Atlantic coast for Florida sites, Pacific for California)")
    print("   - This allows for safe trajectory over water and recovery operations")
    
    # Distance analysis (example values)
    print("\n4. PROXIMITY ANALYSIS (Example for KSC LC-39A):")
    print("   - Distance to coastline: ~2-3 km")
    print("   - Distance to nearest highway: ~5-8 km") 
    print("   - Distance to nearest city: ~10-15 km")
    print("   - Distance to railway: ~3-5 km")
    
    print("\n5. STRATEGIC LOCATION BENEFITS:")
    print("   - Close to coast for safety and recovery")
    print("   - Away from dense population centers")
    print("   - Access to transportation infrastructure")
    print("   - Favorable latitude for orbital mechanics")
    
    return success_rates

In [49]:
analyze_launch_sites()

SpaceX Launch Site Analysis

1. LAUNCH SITE LOCATIONS:
   CCAFS LC-40: Lat 28.562, Long -80.577
   CCAFS SLC-40: Lat 28.563, Long -80.577
   KSC LC-39A: Lat 28.573, Long -80.647
   VAFB SLC-4E: Lat 34.633, Long -120.611

2. SUCCESS RATES BY LAUNCH SITE:
              Total Launches  Successful Launches  Success Rate
Launch Site                                                    
CCAFS LC-40               26                    7         0.269
CCAFS SLC-40               7                    3         0.429
KSC LC-39A                13                   10         0.769
VAFB SLC-4E               10                    4         0.400

3. GEOGRAPHIC INSIGHTS:
   - Proximity to Equator: All sites are relatively close to the equator
     (between 28°N and 35°N latitude) which is advantageous for launches
   - Coastal Location: All sites are very close to coastlines
     (Atlantic coast for Florida sites, Pacific for California)
   - This allows for safe trajectory over water and recovery oper

,Total Launches,Successful Launches,Success Rate
Launch Site,,,
CCAFS LC-40,26,7,0.269
CCAFS SLC-40,7,3,0.429
KSC LC-39A,13,10,0.769
VAFB SLC-4E,10,4,0.400


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
